In [48]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re

In [36]:
df_citylab_rdy = pd.read_csv("Citylab-Berlin_Projekte.csv", sep=";")
df_citylab_rdy.head()

,Quelle,Projektname,Art,Einsatzbereich,Webseite-Link,Organisation,Ansprechperson,Email,Status,Kurzzusammenfassung,Unnamed: 10


***
## Main Page

In [20]:
df_citylab_working = pd.DataFrame()

In [4]:
url = 'https://citylab-berlin.org/de/projects/'
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [33]:
first_chunks = soup.find_all('div', class_='wpgb-masonry')

laufend_chunk = first_chunks[0].find_all('article')
abgeschlossen_chunk = first_chunks[1].find_all('article')

In [147]:
laufend_dict = {}

for tile in laufend_chunk: 
    sub_domain_link = tile.find('a', class_="wpgb-card-layer-link").get('href')
    h3 = tile.find('h3', class_="wpgb-block-3 wpgb-idle-scheme-1")
    if h3:
        textlink = h3.find('a')
        if textlink:
            title = textlink.text
    
    categeory_texts = []
    cat = tile.find('div', class_="wpgb-block-2")
    if cat: 
        categories = cat.find_all('a')
        if categories:
            for category in categories:
                categeory_text = category.text
                categeory_texts.append(categeory_text)
    
    laufend_dict[title] = {
        "sub-link" : sub_domain_link,
        "art" : categeory_texts
    } 

for key in list(laufend_dict.keys()):
    split_string = re.split(r'[\\]', key)
    if split_string:  # Check if the split string has elements
        new_key = split_string[0].strip()
    laufend_dict[new_key] = laufend_dict.pop(key)

laufend_dict

{'BärGPT': {'sub-link': 'https://citylab-berlin.org/de/projects/baergpt/',
  'art': ['Digitale Zusammenarbeit', 'Neue Technologien']},
 'Data Hub Berlin': {'sub-link': 'https://citylab-berlin.org/de/projects/data-hub-berlin/',
  'art': ['Innovative Verwaltung', 'Offene Daten', 'Open Source']},
 'Partizipation Digital': {'sub-link': 'https://citylab-berlin.org/de/projects/partizipation-digital/',
  'art': ['Digitale Zusammenarbeit', 'Innovative Verwaltung']},
 'GovTech TestLAB': {'sub-link': 'https://citylab-berlin.org/de/projects/govtech-testlab/',
  'art': ['Digitale Zusammenarbeit', 'Innovative Verwaltung', 'Open Source']},
 'Stadtlabor2Go': {'sub-link': 'https://citylab-berlin.org/de/projects/stadtlabor2go/',
  'art': ['Digitale Zusammenarbeit',
   'Innovative Verwaltung',
   'Offene Daten, offene Städte']},
 'Fairgnügen': {'sub-link': 'https://citylab-berlin.org/de/projects/fairgnuegen/',
  'art': ['Digitale Zusammenarbeit', 'Innovative Verwaltung', 'Prototyping']},
 'Kiezlabor': {

In [146]:
abgeschlossen_dict = {}

for tile in abgeschlossen_chunk: 
    sub_domain_link = tile.find('a', class_="wpgb-card-layer-link").get('href')
    h3 = tile.find('h3', class_="wpgb-block-3 wpgb-idle-scheme-1")
    if h3:
        textlink = h3.find('a')
        if textlink:
            title = textlink.text

    categeory_texts = []
    cat = tile.find('div', class_="wpgb-block-2")
    if cat: 
        categories = cat.find_all('a')
        if categories:
            for category in categories:
                categeory_text = category.text
                categeory_texts.append(categeory_text)

    abgeschlossen_dict[title] = {
        "sub-link" : sub_domain_link,
        "art" : categeory_texts
    } 

for key in list(abgeschlossen_dict.keys()):
    split_string = re.split(r'[\–\-\:]', key)
    if split_string:  # Check if the split string has elements
        new_key = split_string[0].strip()
    abgeschlossen_dict[new_key] = abgeschlossen_dict.pop(key)

abgeschlossen_dict

{'Digital Vereint': {'sub-link': 'https://citylab-berlin.org/de/projects/digitalvereint/',
  'art': ['Smart Cities für alle']},
 'Quantified Trees (Qtrees)': {'sub-link': 'https://citylab-berlin.org/de/projects/qtrees/',
  'art': ['Energie und Nachhaltigkeit']},
 'Verwaltungsdashboard': {'sub-link': 'https://citylab-berlin.org/de/projects/afs-dashboard/',
  'art': ['Innovative Verwaltung']},
 'Stadtpuls': {'sub-link': 'https://citylab-berlin.org/de/projects/stadtpuls/',
  'art': ['Offene Daten, offene Städte', 'Smart Cities für alle']},
 'Pixelwelt': {'sub-link': 'https://citylab-berlin.org/de/projects/pixelwelt/',
  'art': ['Digitale Zusammenarbeit']},
 'Prototypenwerkstatt': {'sub-link': 'https://citylab-berlin.org/de/projects/prototypenwerkstatt/',
  'art': ['Innovative Verwaltung']},
 'Prozessanalyse Radinfrastruktur (PARI)': {'sub-link': 'https://citylab-berlin.org/de/projects/prozessanalyse-radinfrastruktur-pari/',
  'art': ['Energie und Nachhaltigkeit', 'Offene Daten, offene Stä

*** 
## Subdomains

In [ ]:
import json

abgeschlossen_soup_dict = {}

for key, values in abgeschlossen_dict.items():
    subdomain_url = values['sub-link']
    response_subdomain = requests.get(subdomain_url)
    subsoup = BeautifulSoup(response_subdomain.text, 'html.parser')
    abgeschlossen_soup_dict[key] = subsoup

with open('abgeschlossen_soup_dict.json', 'w', encoding='utf-8') as f:
    json.dump(abgeschlossen_soup_dict, f, ensure_ascii=False, indent=4)

In [ ]:
laufend_soup_dict = {}

for key, values in laufend_dict.items():
    subdomain_url = values['sub-link']
    response_subdomain = requests.get(subdomain_url)
    subsoup = BeautifulSoup(response_subdomain.text, 'html.parser')
    laufend_soup_dict[key] = subsoup

with open('laufend_soup_dict.json', 'w', encoding='utf-8') as f:
    json.dump(laufend_soup_dict, f, ensure_ascii=False, indent=4)

In [ ]:
for key, values in abgeschlossen_dict.items():
    subsoup = abgeschlossen_soup_dict[key]
    
    # Initialize variables to avoid NameError in the matrix
    web_link = None
    kontakt = None
    email = None
    status = None
    text = None
    
    # Extract web link - primary method
    link_header = subsoup.find('h2', class_="wp-block-heading", id="webseite")
    if link_header:
        web_link_container = link_header.find_next('a')
        if web_link_container:
            web_link = web_link_container.get('href')
    
    # Fallback to sub-link if primary method fails
    if not web_link and key in abgeschlossen_dict and 'sub-link' in abgeschlossen_dict[key]:
        web_link = abgeschlossen_dict[key]['sub-link']
    
    kontakt_header = subsoup.find('h2', 
        string=lambda text: text and text.strip().lower() == 'kontakt'
    )
    if kontakt_header:
        # Find the first <p> after the kontakt_header
        kontakt_container = kontakt_header.find_next('p')
        if kontakt_container:
            kontakt = kontakt_container.get_text()

            # Find all <a> tags inside that <p>
            email_links = kontakt_container.find_all('a')

            emails = []
            for link in email_links:
                link_text = link.get_text()
                href = link.get('href', '')

                # Special case for "CityLAB Berlin"
                if link_text == "CityLAB Berlin":
                    emails.append("info@citylab-berlin.org")
                # Extract email from mailto:
                elif href.startswith('mailto:'):
                    email = href.replace('mailto:', '')
                    emails.append(email)
    

    all_h2 = subsoup.find_all('h2')
    status = None
    for h2 in all_h2:
        text = h2.get_text(strip=True)
        if text:  # if the h2 has non-empty text
            status = text
            break  # stop at the first non-empty h2


    content = subsoup.find('div', class_="entry-content")
    if content: 
        warum_text = content.find_next('p')
        if warum_text:
            text = warum_text.get_text(strip=True)
    
    # Update the data structure
    values.update({
        "Web-Link": web_link,
        "Ansprechperson": kontakt,
        "Email": email,
        "Status": status,
        "Kurzzusammenfassung": text
    })

Kontakt text: [email protected]
Emails found: []
Kontakt text: Projektleitung: Julia ZimmermannSenior Product Manager: Myrian Rigal
Emails found: []
Kontakt text: Projektleiter: Tobias Witt
Emails found: []
Kontakt text: Projektverantworlicher: Ingo Hinterding
Emails found: []
Kontakt text: [email protected]
Emails found: []
Kontakt text: Peter Broytman, Koordinierungsstelle Radverkehr
Emails found: []
Kontakt text: Clair Berlin, CityLAB Berlin
Emails found: ['info@citylab-berlin.org']
Kontakt text: Nadine Riede
Emails found: []
Kontakt text: Benjamin Seibel
Emails found: []
Kontakt text: Christian J. Larsen
Emails found: []
Kontakt text: Open Data Informationsstelle
Emails found: []
Kontakt text: Jonas Schorr, Urban Impact
Emails found: []
Kontakt text: Carolin Clausnitzer
Emails found: []


In [ ]:
for key, values in laufend_dict.items():
    subsoup = laufend_soup_dict[key]
    
    # Initialize variables to avoid NameError in the matrix
    web_link = None
    kontakt = None
    email = None
    status = None
    text = None
    
    # Extract web link - primary method
    link_header = subsoup.find('h2', class_="wp-block-heading", id="webseite")
    if link_header:
        web_link_container = link_header.find_next('a')
        if web_link_container:
            web_link = web_link_container.get('href')
    
    # Fallback to sub-link if primary method fails
    if not web_link and key in abgeschlossen_dict and 'sub-link' in abgeschlossen_dict[key]:
        web_link = abgeschlossen_dict[key]['sub-link']
    
    kontakt_header = subsoup.find('h2', 
        string=lambda text: text and text.strip().lower() == 'kontakt'
    )
    if kontakt_header:
        # Find the first <p> after the kontakt_header
        kontakt_container = kontakt_header.find_next('p')
        if kontakt_container:
            kontakt = kontakt_container.get_text()

            # Find all <a> tags inside that <p>
            email_links = kontakt_container.find_all('a')

            emails = []
            for link in email_links:
                link_text = link.get_text()
                href = link.get('href', '')

                # Special case for "CityLAB Berlin"
                if link_text == "CityLAB Berlin":
                    emails.append("info@citylab-berlin.org")
                # Extract email from mailto:
                elif href.startswith('mailto:'):
                    email = href.replace('mailto:', '')
                    emails.append(email)


    status = "Laufend"


    content = subsoup.find('div', class_="entry-content")
    if content: 
        warum_text = content.find_next('p')
        if warum_text:
            text = warum_text.get_text(strip=True)
    
    # Update the data structure
    values.update({
        "Web-Link": web_link,
        "Ansprechperson": kontakt,
        "Email": email,
        "Status": status,
        "Kurzzusammenfassung": text
    })

Kontakt text: Lisa Stubert ([email protected])
Emails found: []
Kontakt text: Markus Sperl (Projektmanager Smart City & Verwaltungsinnovation), E-Mail: [email protected] 
Emails found: []
Kontakt text: Yannick Müller, Strategic Partnerships, Technologiestiftung Berlin
Emails found: []
Kontakt text: Ingo Hinterding (Product Lead / Teamleitung Prototyping & Forschungsprojekte)
Emails found: []
Kontakt text: [email protected]
Emails found: []
Kontakt text: Ingo Hinterding (Product Lead / Teamleitung Prototyping & Forschungsprojekte)
Emails found: []
Kontakt text: Tobias Witt (Service Designer) 
Emails found: []
Kontakt text: Tobias Witt (Service Designer) 
Emails found: []
Kontakt text: Lisa Stubert (Projektleitung Open Data Informationsstelle)
Emails found: []
Kontakt text: Niklas Kossow (Teamleitung Smart City & Verwaltungsinnovation)
Emails found: []
Kontakt text: Yannick Müller (Strategic Partnerships)
Emails found: []
Kontakt text: [email protected]
Emails found: []
Kontakt text: Lis